In [85]:
from pathlib import Path
from medkit.core.text import TextDocument
from medkit.text.segmentation import SentenceTokenizer
from medkit.text.ner import RegexpMatcher, RegexpMatcherRule , RegexpMatcherNormalization
from medkit.text.context import NegationDetector, NegationDetectorRule
from medkit.text.segmentation import SyntagmaTokenizer
from medkit.text.context import FamilyDetector
from unidecode import unidecode
import os
import re
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [86]:
def preprocessing(text):
    
    # Convertir les caractères spéciaux spécifiques avant la conversion en ASCII
    text = re.sub(r'n°', 'numero', text)  # Remplace "n°" par "numero"
    text = re.sub(r'/d°', 'deg', text)  # Remplace "/d°" par "deg"

    # Convertir le texte en ASCII
    ascii_text = unidecode(text)  # Convertit les caractères Unicode en ASCII

    # Normaliser les espaces en remplaçant les espaces multiples par un seul espace
    ascii_text = re.sub(r'\s+', ' ', ascii_text)  # Remplace plusieurs espaces par un seul espace
    return ascii_text

In [87]:
def statut_extraction_tabac(doc):
    statut = "UNKNOWN"
    n_oui = 0
    n_non = 0
    value_is_negated = False
    value_other_detected = False
    
    for ann in doc.anns:
        for attr in ann.attrs:
            if ann.label == "tabagisme":
                if attr.label == "is_negated":
                    value_is_negated = attr.value
                elif attr.label == "other_detected":
                    value_other_detected = attr.value
                break  # On sort de la boucle après avoir récupéré la première valeur


        if value_other_detected:
            continue
        else:
            if value_is_negated:
                n_non += 1
            else:
                n_oui += 1

    if n_non > 0 and n_oui > 0:
        statut = "FUMEUR"
    elif n_non > 0:
        statut = "NON-FUMEUR"
    elif n_oui > 0:
        statut = "FUMEUR"
    return statut

In [128]:
def statut_extraction_alcool(doc):
    statut = "UNKNOWN"
    n_oui = 0
    n_non = 0
    value_is_negated = False
    value_other_detected = False
    count = 0  # Variable de compteur pour suivre le nombre d'occurrences

    for ann in doc.anns:
        for attr in ann.attrs:
            if ann.label == "alcool":
                if attr.label == "is_negated":
                    value_is_negated = attr.value
                elif attr.label == "other_detected":
                    value_other_detected = attr.value
                count += 1  # Incrémenter le compteur

                if count == 2:  # Vérifier si le compteur atteint 2
                    break  # Sortir de la boucle interne une fois que la deuxième valeur est récupérée

        if count == 2:  # Sortir de la boucle externe une fois que la deuxième valeur est récupérée
            if value_other_detected:
                continue
            else:
                if value_is_negated:
                    n_non += 1
                else:
                    n_oui += 1

            if n_non > 0 and n_oui > 0:
                statut = "ALCOOLIQUE"
            elif n_non > 0:
                statut = "NON-ALCOOLIQUE"
            elif n_oui > 0:
                statut = "ALCOOLIQUE"
            break

    return statut

In [129]:
def statut_extraction_situation_familiale(doc):
    ## Initialisation
    situation = "UNKNOWN"
    value_is_negated = False
    value_other_detected = False
    count = 0  # Variable de compteur pour suivre le nombre d'occurrences

    # On parcourt le dico pour analyser chaque annotation trouvée
    for ann in doc.anns:
        if ann.label == "situation":
            for attr in ann.attrs:
                if attr.label == "is_negated":
                    value_is_negated = attr.value
                elif attr.label == "other_detected":
                    value_other_detected = attr.value

            # Si l'entité trouvée ne concerne pas le patient (other_detected == True),
            # on passe à l'annotation suivante
            if value_other_detected == True:
                continue
            else:
                situation = ann.text.lower()
                ## NORMALISATION: Seul, pas seul ou inconnu
                if re.search(r"\bmarie[e]?\b", situation):
                    situation = "PAS SEUL"
                elif re.search(r"\bcelibataire\b", situation):
                    situation = "SEUL"
                elif re.search(r"\bdivorce[e]?\b", situation):
                    situation = "SEUL"
                elif re.search(r"\bveuf\b", situation):
                    situation = "SEUL"
                elif re.search(r"\bveuve\b", situation):
                    situation = "SEUL"
                elif re.search(r"\bpacse[e][s]?\b", situation):
                    situation = "PAS SEUL"
                elif re.search(r"\bconcubinage\b", situation):
                    situation = "PAS SEUL"
                elif re.search(r"\b(vit|habite)\sseul(e)?\b", situation):
                    situation = "SEUL"

                # Si il y a une négation
                if value_is_negated == True:
                    # On inverse statut_marital
                    if situation == "SEUL":
                        situation = "PAS SEUL"
                    else:
                        situation = "SEUL"

                if count == 3:  # Sortir de la boucle externe une fois que la troisième valeur est récupérée
                    break
    return situation  

In [130]:
def clinical_case_recovery(output_folder):
    # On récupère tous les fichiers texte dans le dossier
    txt_files = [f for f in os.listdir(output_folder) if f.endswith('.txt')]

    # On trie les fichiers par ordre alphabétique.
    txt_files_sorted = sorted(txt_files)
    textes = {}  # dictionnaire de tous les cas cliniques

    # On ouvre et on extrait les textes dans textes
    for i in range(len(txt_files_sorted)):
        file_path = os.path.join(output_folder, txt_files_sorted[i])
        with open(file_path, 'r') as f:
            text = f.read()
        textes[txt_files_sorted[i]] = text
    return textes

## LES NEG DETECTOR

In [131]:

def neg_detector_tabac():

    ### QUELQUES REGEX NEGATION

    neg_rules = [       

    NegationDetectorRule(regexp=r"\bne\s*(semble|consomme|prend|fume)\s*pas"),
    NegationDetectorRule(regexp=r"jamais"),
    NegationDetectorRule(regexp=r"\bni\b"),
    NegationDetectorRule(regexp=r"\bnon\s+\b"),
    NegationDetectorRule(regexp=r"Tabac\s*[=:]?\s*0"),
    NegationDetectorRule(regexp=r"(pas|ni|ou)\s+de\s+(consommation\s+de\s+)?taba"),
    NegationDetectorRule(regexp=r"pas\s+d\'intoxication\s+tabagi"),
    NegationDetectorRule(regexp=r"0 tabac"),

    ]
    
    neg_detector = NegationDetector(output_label="is_negated", rules=neg_rules)
    return neg_detector

In [132]:
def neg_detector_alcool():

    ### QUELQUES REGEX NEGATION

    neg_rules = [

        NegationDetectorRule(regexp=r"ne\s*boit\s*pas"),
        NegationDetectorRule(regexp=r"\bne/s*consomme/s*pas\b"),
        NegationDetectorRule(regexp=r"\bni\b"),
        NegationDetectorRule(regexp=r"\bpas\b"),
        NegationDetectorRule(regexp=r"\becarte\b"),
    ]
    neg_detector = NegationDetector(output_label="is_negated", rules=neg_rules)
    return neg_detector

In [133]:
def neg_detector_situation_familiale():

### QUELQUES REGEX NEGATION

    neg_rules = [

        NegationDetectorRule(regexp=r"\bn'est pas\b"),
        NegationDetectorRule(regexp=r"\bne vit pas\b"),
        NegationDetectorRule(regexp=r"\bn'habite pas\b"),
        NegationDetectorRule(regexp=r"\bni\b"),
    ]
    neg_detector = NegationDetector(output_label="is_negated", rules=neg_rules)
    return neg_detector

## REGEX

In [134]:
regexp_rules_tabac = [
    RegexpMatcherRule(regexp=r"\bcigare(tte)?[s]?\b", label="tabagisme", exclusion_regexp ="en bout de cigare"),
    RegexpMatcherRule(regexp=r"\bfume(e)?\b", label="tabagisme", exclusion_regexp = "residu(s)?/s+de/s+fumee(s)?"),
    RegexpMatcherRule(regexp=r"\btaba(c|gisme|gique)\b", label="tabagisme"),
    RegexpMatcherRule(regexp=r"fumeur|fumeuse", label="tabagisme"),
    RegexpMatcherRule(regexp=r"fumait", label="tabagisme"),

]
regexp_rules_alcool = [
    RegexpMatcherRule(regexp=r"\balcool(o)?\b", label="alcool"),
    RegexpMatcherRule(regexp=r"\bboit\b", label="alcool"),
    RegexpMatcherRule(regexp=r"\bbuvait\b", label="alcool"),
    RegexpMatcherRule(regexp=r"\balcoolique\b", label="alcool"),
    RegexpMatcherRule(regexp=r"\balcoolisme\b", label="alcool"),
]

regexp_rules_familial = [
    RegexpMatcherRule(regexp=r"\bmarie[e]?\b", label="situation"),
    RegexpMatcherRule(regexp=r"\bcelibataire\b", label="situation"),
    RegexpMatcherRule(regexp=r"\bdivorce[e]?\b", label="situation"),
    RegexpMatcherRule(regexp=r"\bveuf\b", label="situation"),
    RegexpMatcherRule(regexp=r"\bveuve\b", label="situation"),
    RegexpMatcherRule(regexp=r"\bpacse[e][s]?\b", label="situation"),
    RegexpMatcherRule(regexp=r"\bconcubinage\b", label="situation"),
    RegexpMatcherRule(regexp=r"\b(vit|habite)\sseul(e)?\b", label="situation"),
]

## EXTRACTION STATUTS

In [135]:
def extraction_finale(clinical_case_repo):
    # Listes pour stocker les données
    data_tabac = []
    data_alcool = []
    data_situation = []
    data = []
    docs = []
    # On charge les cas cliniques dans un dico {nom fichier: cas clinique}
    clinical_cases_dico = clinical_case_recovery(clinical_case_repo)

    for fichier, clinical_case in clinical_cases_dico.items():
        clinical_case = preprocessing(clinical_case) # prétraitement du texte
        doc = TextDocument(text=clinical_case)

        ## On sépare le texte en phrases
        sent_tokenizer = SentenceTokenizer(
            output_label="sentence",
            punct_chars=[".", "?", "!"],
        )
        sentences = sent_tokenizer.run([doc.raw_segment])

        ## On sépare les phrases en syntagmas
        synt_tokenizer = SyntagmaTokenizer(
            output_label="sentence",
            separators=[r"\bmais\b", r"\bet\b"],
        )
        syntagmas = synt_tokenizer.run(sentences)

        # Création de l'objet neg detector
        neg_detector_tabac_obj = neg_detector_tabac()
        neg_detector_alcool_obj = neg_detector_alcool()
        neg_detector_statut_familial_obj = neg_detector_situation_familiale()

        # On applique neg detector aux syntagmas
        neg_detector_tabac_obj.run(syntagmas)
        neg_detector_alcool_obj.run(syntagmas)
        neg_detector_statut_familial_obj.run(syntagmas)

        # On applique family detector aux syntagmas
        family_detector = FamilyDetector(output_label='other_detected')
        family_detector.run(syntagmas)

        # CREATION OF ENTITIES
        regexp_matcher_tabac = RegexpMatcher(rules=regexp_rules_tabac, attrs_to_copy=["is_negated", "other_detected"])
        regexp_matcher_alcool = RegexpMatcher(rules=regexp_rules_alcool, attrs_to_copy=["is_negated", "other_detected"])
        regexp_matcher_familial = RegexpMatcher(rules=regexp_rules_familial, attrs_to_copy=["is_negated", "other_detected"])

        entities_tabac = regexp_matcher_tabac.run(syntagmas)
        entities_alcool = regexp_matcher_alcool.run(syntagmas)
        entities_familial = regexp_matcher_familial.run(syntagmas)

        for entity in entities_tabac:
            doc.anns.add((entity))
           
        for entity in entities_alcool:
            doc.anns.add((entity))
        
        for entity in entities_familial:
            doc.anns.add((entity))
        
        docs.append(doc)
               
        tabagisme = statut_extraction_tabac(doc)
        alcool = statut_extraction_alcool(doc)
        situation = statut_extraction_situation_familiale(doc)

        # Remplissage de data
        data_tabac.append([fichier, clinical_case, tabagisme])
        data_alcool.append([fichier, clinical_case, alcool])
        data_situation.append([fichier, clinical_case, situation])
        data.append([fichier, clinical_case, tabagisme,alcool,situation])

    df_tabac = pd.DataFrame(data_tabac, columns=["nom fichier", "cas clinique", "tabagisme"])
    df_alcool = pd.DataFrame(data_alcool, columns=["nom fichier", "cas clinique", "alcool"])
    df_situation = pd.DataFrame(data_situation, columns=["nom fichier", "cas clinique", "situation"])
    df = pd.DataFrame(data, columns=["nom fichier", "cas clinique", "tabagisme", "alcool", "situation"])
    
    
    combined_df = df_tabac
    combined_df = pd.concat([df_tabac, df_alcool, df_situation], axis=1)
    combined_df = combined_df.loc[:, ~combined_df.columns.duplicated()]
    combined_df = combined_df.dropna()
    
    return df,docs

In [136]:
# Listes pour stocker les données
data_tabac = []
data_alcool = []
data_situation = []


df,docs = extraction_finale("clinical_case2")
print("fin")

fin


In [137]:
df

nom fichier   
0      filehtml-1-cas.txt  \
1    filehtml-146-cas.txt   
2     filehtml-21-cas.txt   
3     filehtml-23-cas.txt   
4     filehtml-24-cas.txt   
..                    ...   
712    filepdf-97-cas.txt   
713  filepdf-98-1-cas.txt   
714  filepdf-98-2-cas.txt   
715  filepdf-98-3-cas.txt   
716    filepdf-99-cas.txt   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 

# PREPARATION DES FICHIERS A ANNOTER 

## PREPARATION ANNOTATION TABAC

In [42]:
combined_df['tabagisme'].value_counts()

tabagisme
UNKNOWN       665
FUMEUR         41
NON-FUMEUR     11
Name: count, dtype: int64

In [28]:
combined_df['tabagisme'].value_counts()

tabagisme
UNKNOWN       665
FUMEUR         37
NON-FUMEUR     15
Name: count, dtype: int64

In [61]:
df_non_fumeur = combined_df[combined_df["tabagisme"] == "NON-FUMEUR"].head(17)
df_fumeur = combined_df[combined_df["tabagisme"] == "FUMEUR"].head(17)
df_unknown = combined_df[combined_df["tabagisme"] == "UNKNOWN"].head(17)

In [62]:
df_statut_tabac = pd.concat([df_non_fumeur, df_fumeur, df_unknown], ignore_index=True)

In [72]:
df_statut_tabac.shape

(51, 3)

In [64]:
df_statut_tabac['statut tabagisme'].value_counts()

statut tabagisme
NON-FUMEUR    17
FUMEUR        17
UNKNOWN       17
Name: count, dtype: int64

In [74]:
# Récupération dans un dossier des fichiers textes à annoter

import os
import shutil

chemin_dossier_source = "clinical_case2"
#chemin_dossier_destination = "51_fichiers_annotation_tabac"
# On crée le dossier destination si il n'existe pas 
os.makedirs(chemin_dossier_destination, exist_ok=True)

for nom_fichier in df_statut_tabac['nom fichier']:
    chemin_fichier_source = os.path.join(chemin_dossier_source, nom_fichier)
    chemin_fichier_destination = os.path.join(chemin_dossier_destination, nom_fichier)
    shutil.copy(chemin_fichier_source, chemin_fichier_destination)


## On fait tourner le code sur ces fichiers uniquement et on extrait seulement le statut tabagique

In [105]:
# Listes pour stocker les données
data_tabac = []

### QUELQUES REGEX RECHERCHE ENTITES
regexp_rules_tabac = [
    RegexpMatcherRule(regexp=r"\bcigare(tte)?[s]?\b", label="statut_tabagisme", exclusion_regexp ="en bout de cigare"),
    RegexpMatcherRule(regexp=r"\bfume(e)?[s]?\b", label="statut_tabagisme"),
    RegexpMatcherRule(regexp=r"\btaba(c|gisme|gique)\b", label="statut_tabagisme"),
    RegexpMatcherRule(regexp=r"fumeur|fumeuse", label="statut_tabagisme"),
]

docs_medkit = []
path = "50_fichiers_annotation_tabac"
# On charge les cas cliniques dans un dico {nom fichier: cas clinique}
clinical_cases_dico = clinical_case_recovery(path)

for fichier, clinical_case in clinical_cases_dico.items():
    clinical_case = preprocessing(clinical_case)
    doc = TextDocument(text=clinical_case)

    ## On sépare le texte en phrases
    sent_tokenizer = SentenceTokenizer(
        output_label="sentence",
        punct_chars=[".", "?", "!"],
    )
    sentences = sent_tokenizer.run([doc.raw_segment])

    ## On sépare les phrases en syntagmas
    synt_tokenizer = SyntagmaTokenizer(
        output_label="sentence",
        separators=[r"\bmais\b", r"\bet\b"],
    )
    syntagmas = synt_tokenizer.run(sentences)

    # Création de l'objet neg detector
    neg_detector_tabac_obj = neg_detector_tabac()

    # On applique neg detector aux syntagmas
    neg_detector_tabac_obj.run(syntagmas)

    # On applique family detector aux syntagmas
    family_detector = FamilyDetector(output_label='other_detected')
    family_detector.run(syntagmas)

    # CREATION OF ENTITIES
    regexp_matcher_tabac = RegexpMatcher(rules=regexp_rules_tabac, attrs_to_copy=["is_negated", "other_detected"])

    entities_tabac = regexp_matcher_tabac.run(syntagmas)

    for entity in entities_tabac:
        doc.anns.add((entity))
    docs_medkit.append(doc)

    statut_tabagisme = statut_extraction_tabac(doc)

    # Remplissage de data
    data_tabac.append([fichier, clinical_case, statut_tabagisme])


df_tabac = pd.DataFrame(data_tabac, columns=["nom fichier", "cas clinique", "statut tabagisme"])
df_tabac = df_tabac.dropna()

In [106]:
df_tabac

nom fichier   
0      filehtml-1-cas.txt  \
1    filehtml-146-cas.txt   
2     filehtml-21-cas.txt   
3     filehtml-23-cas.txt   
4     filehtml-24-cas.txt   
5     filehtml-29-cas.txt   
6     filehtml-30-cas.txt   
7     filehtml-31-cas.txt   
8     filehtml-32-cas.txt   
9     filehtml-34-cas.txt   
10      filepdf-1-cas.txt   
11     filepdf-10-cas.txt   
12    filepdf-100-cas.txt   
13    filepdf-101-cas.txt   
14    filepdf-102-cas.txt   
15  filepdf-103-1-cas.txt   
16  filepdf-103-2-cas.txt   
17  filepdf-103-3-cas.txt   
18    filepdf-104-cas.txt   
19    filepdf-105-cas.txt   
20    filepdf-130-cas.txt   
21    filepdf-138-cas.txt   
22     filepdf-16-cas.txt   
23  filepdf-171-4-cas.txt   
24    filepdf-193-cas.txt   
25    filepdf-205-cas.txt   
26    filepdf-221-cas.txt   
27    filepdf-223-cas.txt   
28    filepdf-239-cas.txt   
29  filepdf-263-1-cas.txt   
30  filepdf-263-2-cas.txt   
31  filepdf-263-3-cas.txt   
32    filepdf-272-cas.txt   
33    filepdf-273-cas.txt   
34     filepdf-28-cas.txt   
35     filepdf-29-cas.txt   
36    filepdf-454-cas.txt   
37    filepdf-474-cas.txt   
38     filepdf-50-cas.txt   
39  filepdf-508-1-cas.txt   
40     filepdf-52-cas.txt   
41    filepdf-680-cas.txt   
42    filepdf-743-cas.txt   
43    filepdf-750-cas.txt   
44    filepdf-756-cas.txt   
45    filepdf-757-cas.txt   
46    filepdf-836-cas.txt   
47  filepdf-840-1-cas.txt   
48    filepdf-843-cas.txt   
49  filepdf-883-1-cas.txt   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [107]:
len(docs_medkit)

50

## EVALUATION: COMPARAISON ENTRE ANNOTATION BRAT ET MEDKIT

In [108]:
# On récupère les annotations brat

from medkit.io.brat import BratInputConverter

# Define Input Converter 
brat_converter = BratInputConverter()

path= "/home/mhassani/Documents/Stage/env/brat-master/data/collection"

# Load brat into a list of documents
docs_brat = brat_converter.load(dir_path = path)
len(docs_brat)

50

In [109]:
len(docs_medkit)

50

In [110]:
type(docs_brat[0])

medkit.core.text.document.TextDocument

In [111]:
type(docs_medkit[0])

medkit.core.text.document.TextDocument

In [112]:
docs_medkit[0].anns

TextAnnotationContainer(doc_id='c47dc652-14e1-11ee-a77d-5d93ca66babf', anns=[Entity(uid='c47dc6c1-14e1-11ee-a77d-5d93ca66babf', label='statut_tabagisme', attrs=EntityAttributeContainer(ann_id='c47dc6c1-14e1-11ee-a77d-5d93ca66babf', attrs=[Attribute(label='is_negated', value=False, metadata={}, uid='c47dc6c2-14e1-11ee-a77d-5d93ca66babf'), Attribute(label='other_detected', value=False, metadata={}, uid='c47dc6c3-14e1-11ee-a77d-5d93ca66babf')]), metadata={'rule_id': 2, 'version': None}, keys=set(), spans=[Span(start=239, end=248)], text='tabagique'), Entity(uid='c47dc6c4-14e1-11ee-a77d-5d93ca66babf', label='statut_tabagisme', attrs=EntityAttributeContainer(ann_id='c47dc6c4-14e1-11ee-a77d-5d93ca66babf', attrs=[Attribute(label='is_negated', value=True, metadata={'rule_id': 7}, uid='c47dc6c5-14e1-11ee-a77d-5d93ca66babf'), Attribute(label='other_detected', value=False, metadata={}, uid='c47dc6c6-14e1-11ee-a77d-5d93ca66babf')]), metadata={'rule_id': 2, 'version': None}, keys=set(), spans=[Span

In [113]:
from medkit.core.text import Span
from medkit.core.text import Entity

## On met sous la bonne forme les annotations medkit
pred_ents = []

for doc in docs_medkit:
    entities = []
    for entity in doc.anns:
        entity_spans = [Span(start=span.start, end=span.end) for span in entity.spans]
        entity_obj = Entity(label=entity.label, spans=entity_spans, text=entity.text)
        entities.append(entity_obj)
    
    pred_ents.append(entities)
pred_ents[0:3]

[[Entity(uid='ca8ff75e-14e1-11ee-a77d-5d93ca66babf', label='statut_tabagisme', attrs=EntityAttributeContainer(ann_id='ca8ff75e-14e1-11ee-a77d-5d93ca66babf', attrs=[]), metadata={}, keys=set(), spans=[Span(start=239, end=248)], text='tabagique'),
  Entity(uid='ca8ff75f-14e1-11ee-a77d-5d93ca66babf', label='statut_tabagisme', attrs=EntityAttributeContainer(ann_id='ca8ff75f-14e1-11ee-a77d-5d93ca66babf', attrs=[]), metadata={}, keys=set(), spans=[Span(start=423, end=432)], text='tabagisme')],
 [],
 []]

In [116]:
from medkit.core.text import TextDocument, Entity, Span
from medkit.text.metrics.ner import SeqEvalEvaluator

# define a evaluator using `iob2` as tagging scheme
evaluator = SeqEvalEvaluator(tagging_scheme="iob2")
metrics = evaluator.compute(documents=docs_brat, predicted_entities=pred_ents)

In [117]:
for metric, value in metrics.items():
    print(f"{metric}: {value}")

overall_precision: 0.7714285714285715
overall_recall: 0.75
overall_f1-score: 0.7605633802816902
overall_support: 36
overall_acc: 0.9997296744885243
statut_tabagisme_precision: 0.7714285714285715
statut_tabagisme_recall: 0.75
statut_tabagisme_f1-score: 0.7605633802816902
statut_tabagisme_support: 36


In [102]:
for i in pred_ents:
    print(i)
    print("\n")

[Entity(uid='6e9611d8-14bc-11ee-a77d-5d93ca66babf', label='statut_tabagisme', attrs=EntityAttributeContainer(ann_id='6e9611d8-14bc-11ee-a77d-5d93ca66babf', attrs=[]), metadata={}, keys=set(), spans=[Span(start=239, end=248)], text='tabagique'), Entity(uid='6e9611d9-14bc-11ee-a77d-5d93ca66babf', label='statut_tabagisme', attrs=EntityAttributeContainer(ann_id='6e9611d9-14bc-11ee-a77d-5d93ca66babf', attrs=[]), metadata={}, keys=set(), spans=[Span(start=423, end=432)], text='tabagisme'), Entity(uid='6e9611da-14bc-11ee-a77d-5d93ca66babf', label='statut_tabagisme', attrs=EntityAttributeContainer(ann_id='6e9611da-14bc-11ee-a77d-5d93ca66babf', attrs=[]), metadata={}, keys=set(), spans=[Span(start=451, end=464)], text='paquets-annee')]


[]


[]


[]


[Entity(uid='6e9611db-14bc-11ee-a77d-5d93ca66babf', label='statut_tabagisme', attrs=EntityAttributeContainer(ann_id='6e9611db-14bc-11ee-a77d-5d93ca66babf', attrs=[]), metadata={}, keys=set(), spans=[Span(start=397, end=401)], text='fume')]


[]



In [77]:
for entity in docs_medkit[0].anns.get_entities():
    print(f"label={entity.label}, spans={entity.spans}, text={entity.text!r}")

label=statut_tabagisme, spans=[Span(start=239, end=248)], text='tabagique'
label=statut_tabagisme, spans=[Span(start=423, end=432)], text='tabagisme'
label=statut_tabagisme, spans=[Span(start=451, end=464)], text='paquets-annee'


In [78]:
for entity in docs_brat[0].anns.get_entities():
    print(f"label={entity.label}, spans={entity.spans}, text={entity.text!r}")

label=statut_tabagisme, spans=[Span(start=239, end=248)], text='tabagique'
label=statut_tabagisme, spans=[Span(start=423, end=432)], text='tabagisme'
label=statut_tabagisme, spans=[Span(start=448, end=465)], text='21 paquets-années'


## PREPARATION ANNOTATION ALCOOL

In [19]:
df_non_alcool = combined_df[combined_df["statut alcool"] == "NON-AlCOOLIQUE"]
df_alcool = combined_df[combined_df["statut alcool"] == "ALCOOLIQUE"].head(20)
df_unknown = combined_df[combined_df["statut alcool"] == "UNKNOWN"].head(24)

In [20]:
df_statut_alcool = pd.concat([df_non_alcool, df_alcool, df_unknown], ignore_index=True)

In [21]:
df_statut_alcool['statut alcool'].value_counts()

statut alcool
UNKNOWN           24
ALCOOLIQUE        20
NON-AlCOOLIQUE     6
Name: count, dtype: int64

In [22]:
df_statut_alcool.shape

(50, 5)

## PREPARATION ANNOTATION SITUATION FAMILIALE

In [23]:
combined_df['statut familial'].value_counts()

statut familial
UNKNOWN     696
SEUL         11
PAS SEUL     10
Name: count, dtype: int64

In [24]:
df_seul = combined_df[combined_df["statut familial"] == "SEUL"]
df_pas_seul = combined_df[combined_df["statut familial"] == "PAS SEUL"]
df_unknown = combined_df[combined_df["statut familial"] == "UNKNOWN"].head(29)

In [25]:
df_statut_familial = pd.concat([df_seul, df_pas_seul, df_unknown], ignore_index=True)

In [26]:
df_statut_familial['statut familial'].value_counts()

statut familial
UNKNOWN     29
SEUL        11
PAS SEUL    10
Name: count, dtype: int64

In [27]:
df_statut_familial.shape

(50, 5)